# ルートを評価するために解くLPについて
- ルートが1つ与えられた後、そのルートを評価するためにLPを解く(ルートが実行可能であるとは限らない)

<!--- 各制約の違反度を変数とし、それらに重みをかけて足し合わせた関数の最小化問題とする-->
- 車両が各顧客へ到着する時刻を変数とし、その合計を最小化する問題とする
- 制約は、容量制約と時間枠制約とする
    - 容量制約は、ある区間における2つの関数の積分値(面積)の大小を比較するという形で表す(車両の積荷の量を表す区分線形関数の\[x0, xn\]までの積分値と最大容量を表す線形関数の\[x0, xn\]までの積分値)
    - 時間枠制約は、通常のVRPの定式化と同様に表す


# 前準備

## 問題例の読み込み
使用した問題例 : https://neo.lcc.uma.es/vrp/vrp-instances/description-for-files-of-solomons-instances/

In [ ]:
class Customer():
    def __init__(self, x, y, d, e, l, s):
        self.x = x
        self.y = y
        self.d = d
        self.e = e
        self.l = l
        self.s = s

In [ ]:
import glob

files = glob.glob("./solomon_25/*")
for file in files:
    print(file)

In [ ]:
# 問題例の選択
instance = files[0]

In [ ]:
Customers = {}
with open(instance, mode="r") as f:
    for index,line in enumerate(f):
        #print(line)
        l = []
        if index==4:
            line_s = [s.strip("\n") for s in line.split("\t")][0]
            for s in line_s.split(" "):
                if s!="":
                    l.append(int(s))
            K, Q = l
        elif 9<=index<=34:
            line_s = [s.strip("\n") for s in line.split("\t")][0]
            for s in line_s.split(" "):
                if s!="":
                    l.append(int(s))
            Customers[l[0]] = Customer(*l[1:])

In [ ]:
Customers["depot"] = Customer(0,0,0,0,1000,0)

In [ ]:
print("\t", list(vars(Customers["depot"]).keys()))
for i in Customers.keys():
    print(i, end="\t")
    for key, val in vars(Customers[i]).items():
        print(val, end=" ")
    print()

## ルートの作成

In [ ]:
# 入力
## 顧客
C={} # 客の座標を保存する辞書
TW={} # 客の時間枠を保存する辞書
demand={} # 客の要求量(正の値は集荷，負の値は配達)
S={} # 客のサービス時間

for i in Customers:
    x, y = Customers[i].x, Customers[i].y
    C[i] = (x,y)
    e, l = Customers[i].e, Customers[i].l
    TW[i] = (e,l)
    demand[i] = (Customers[i].d, )
    S[i] = Customers[i].s
    """name="c"+str(i)
    x, y = Customers[i].x, Customers[i].y
    C[name] = (x,y)
    e, l = Customers[i].e, Customers[i].l
    TW[name] = (e,l)
    demand[name] = (Customers[i].d, )
    S[name] = Customers[i].s"""
#C["depot"]=(0,0)

## 車両
M = K//2 # number of vehicles
capacities = [(Q*2, ) for k in range(M)] # capacity of vehicle

In [ ]:
print("Customer",C)
print("Time Window",TW)
print("Demand",demand)
print("Service Time",S)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(7,7))

G=nx.DiGraph()
nx.draw_networkx(G,pos=C,nodelist=[i for i in C if i != "depot"],node_color="y",node_size=50,with_labels=True,edge_color="k",width=1)
nx.draw_networkx(G,pos=C,nodelist=["depot"],node_color="blue",node_shape='s',alpha=0.5,node_size=100,with_labels=True,edge_color="k",width=1)

plt.show()

In [ ]:
# 距離関数の定義
#def Distance(t1,t2):
#    return ((t1[0]-t2[0])**2+(t1[1]-t2[1])**2)**(0.5)
def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(0.5)

# ソルバーの読み込み
import sys
sys.path.append('..')

#from vrplib.vrp_d_1m1_t_model import *
import vrplib.vrp_d_1m1_t_model as vrp

# ソルバーの実行
model = vrp.Model(file.split("/")[-1].split(".")[0]) # モデルインスタンスの生成

## 客インスタンスの生成
for i in C:
    if i == "depot":
        continue
    model += vrp.Customer(i,demand=demand[i],timewindow=TW[i],servicetime=S[i])

## 車両インスタンスの生成
for k in range(M):
    model += vrp.Vehicle("v"+str(k),capacity=capacities[k])
    
## 枝インスタンスの生成
for i in C:
    for j in C:
        if i!=j:
            dist = time = distance(*C[i],*C[j])
            model += vrp.Edge(i,j,dist,time)

In [ ]:
## 最適化の実行
obj=model.optimize(IterLimit=10000,TimeLimit=10,Verbose=False,OutputFlag=False)

## 得られた解の表示
print("objective value =",obj)
for v in model.vehiclesL:
    print()
    print(v)
    print(list(map(lambda x:x.name,v.routing[1:-1])))    
    for iv in map(lambda x:x.name,v.routing[1:-1]):
        print(iv)

## ルートを1つ選ぶ

In [ ]:
# Vehicle 6の巡回路を選んでいる
tour = [12, 3, 24, 25]#+["depot"]

## その他作業

In [ ]:
# 使用した問題例の名称を保存
tmp = []
for s in instance[-1:0:-1]:
    if s=='/':
        break
    tmp.append(s)
instance_name = "".join(tmp[-1:0:-1])
print(instance_name)

# 前準備

## instances.pyからインスタンスを得る

In [1]:
import instances
Customers = instances.Customers
tour = instances.tour

In [2]:
instance_name = "C" + str(len(Customers)-1)

# 問題を解く
2つの手法に対し、計算時間を比較

## 入力する行列、ベクトルの作成

In [3]:
def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(0.5)

In [4]:
def make_preceding_constr(tour, Customers, Ax, Ap, b, c):
    # ルート内の顧客の順序に関する制約
    for index, i in enumerate(tour):
        try:
            i_next = tour[index+1]
        except:
            i_next = "depot"
            continue
        Ax.append([1 if target==i else -1 if target==i_next else 0 for target in tour])
        Ap.append([0 for _ in range(len(tour))])
        b.append(-distance(Customers[i].x, Customers[i].y, Customers[i_next].x, Customers[i_next].y))
    return Ax, Ap, b, c

In [5]:
def make_tw_constr(tour, Customers, Ax, Ap, b, c):
    # 時間枠制約
    for index, i in enumerate(tour):
        Ax.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        b.append(-Customers[i].e)
        Ax.append([1 if i==target else 0 for index_, target in enumerate(tour)])
        Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        b.append(Customers[i].l)
    return Ax, Ap, b, c

In [6]:
def make_inputs(tour, Customers):
    import numpy as np
    Ax, Ap, b, c = [], [], [], [1 for _ in range(len(tour))]
    # ルート内の顧客の順序に関する制約
    Ax, Ap, b, c = make_preceding_constr(tour, Customers, Ax, Ap, b, c)
    # 時間枠制約
    Ax, Ap, b, c = make_tw_constr(tour, Customers, Ax, Ap, b, c)
    # ndarrayに変換
    Ax = np.array(Ax)
    Ap = np.array(Ap)
    b = np.array(b)
    c = np.array(c)
    return Ax, Ap, b, c

## 主問題を解く関数の定義
与えられた定数を元にLPのモデルを作成した上でそれを解き、最適解を返す関数

In [7]:
def solve_primal(Ax, Ap, b, c, instance_name, num_vars):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    # 定数を設定　←　入力として与えられる
    # 変数を設定
    """
    x_i : 顧客iへ車両が到着する時刻を表す変数
    p_i : 顧客iの時間枠の違反度を表す変数
    """
    x = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="x")
    p = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="p")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(c.T @ p, sense=gp.GRB.MINIMIZE)
    
    # 制約条件を設定
    m.addConstr(Ax @ x + Ap @ p <= b, name="c")

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # 最適化
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"車両が顧客{i}に到着する時刻は、{x[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')
        
    return m

## 双対問題を解く関数の定義

In [8]:
def solve_dual(Ax, Ap, b, c, instance_name, num_vars, basis_like):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    
    # 変数を設定
    """
    y_i : 主問題における制約式iの潜在価値
    """
    y = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="y")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(-1 * b.T @ y, sense=gp.GRB.MAXIMIZE)
    
    # 制約条件を設定
    m.addConstr(Ax.T @ y >= 0, name="c1")
    m.addConstr(Ap.T @ y + c >= 0, name="c2")
    m.addConstr(y >= 0, name="c3")

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # ホットスタートの使用
    for i in range(num_vars):
        y[i].PStart = basis_like[i]
    
    # 最適化
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"主問題における制約{i}の潜在価格は、{y[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')
        
    return m

## ①全体を1つのLPとして解くveb.

### 全体のPrimalを解く

In [9]:
Ax, Ap, b, c = make_inputs(tour, Customers)

In [10]:
# Gurobiによって最適解を求める
whole_x = solve_primal(Ax, Ap, b, c, instance_name+"whole_x", len(tour))

Using license file /Users/okamoto/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2624 rows, 1750 columns and 5248 nonzeros
Model fingerprint: 0xe6d7ed92
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 2606 rows and 1733 columns
Presolve time: 0.01s
Presolved: 18 rows, 17 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0056208e+07   5.955573e+01   0.000000e+00      0s
       2    2.0056287e+07   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds
Optimal objective  2.005628700e+07
車両が顧客0に到着する時刻は、[0.]
車両が顧客1に到着する時刻は、[35.77708764]
車両が顧客2に到着する時刻は、[108.01281287]
車両が顧客3に到着する時刻は、[136.2440013]
車両が顧客4に到着する時刻は、[175.53776671]
車両が顧客5に到着する時刻は、[221.07895686]
車両が顧客6に到着する時刻は、[235.22109248]
車両が顧客7に到着する時刻は、[253.578

In [11]:
for var in whole_x.getVars():
    print(var.varName, var.X)

x[0] 0.0
x[1] 35.77708763999664
x[2] 108.01281287342944
x[3] 136.24400130041565
x[4] 175.53776670919265
x[5] 221.07895685613545
x[6] 235.2210924798664
x[7] 253.57865223055222
x[8] 323.8139709811637
x[9] 355.2146079173789
x[10] 396.939900007429
x[11] 451.4284309107301
x[12] 483.6774619039243
x[13] 574.682956243552
x[14] 669.2398174435727
x[15] 718.2806167845297
x[16] 771.5065503132147
x[17] 798.4137984073622
x[18] 866.2149783380978
x[19] 896.2981962510804
x[20] 899.9037475265444
x[21] 980.2654299380022
x[22] 1003.9719691202615
x[23] 1015.6338729099522
x[24] 1028.6722777203574
x[25] 1076.2117350163762
x[26] 1109.9459906032423
x[27] 1166.8318412165538
x[28] 1230.9021151351192
x[29] 1344.7133623824805
x[30] 1440.2382282551944
x[31] 1442.474296232694
x[32] 1514.585321741974
x[33] 1566.9112235497785
x[34] 1581.0533591735093
x[35] 1657.2110902321483
x[36] 1717.7503337028095
x[37] 1764.9520285876333
x[38] 1816.9231744283784
x[39] 1883.2104281050847
x[40] 1930.917869872591
x[41] 2020.4723225815

p[123] 6102.439349806473
p[124] 5777.841104057464
p[125] 6057.458408150834
p[126] 6312.358352533464
p[127] 6227.820465045817
p[128] 6684.413943017714
p[129] 6311.802572673497
p[130] 6778.097230365332
p[131] 6728.933692153631
p[132] 6356.780391120909
p[133] 6592.2784531072975
p[134] 6215.876659349253
p[135] 6599.808424622231
p[136] 6529.816898589958
p[137] 6732.126084533987
p[138] 6745.164489344392
p[139] 6782.447668995361
p[140] 7070.526561766682
p[141] 6610.592706067649
p[142] 7372.946467489282
p[143] 7405.499169683016
p[144] 6908.777990279115
p[145] 6898.049877521473
p[146] 7733.274562002
p[147] 7546.415883024345
p[148] 7548.797379848891
p[149] 7470.078137194596
p[150] 7106.5567042482035
p[151] 7505.5567042482035
p[152] 7314.723140074168
p[153] 7726.490198183454
p[154] 7698.739007680268
p[155] 7391.339602913091
p[156] 8039.464386649468
p[157] 7642.982804131534
p[158] 7804.549591650617
p[159] 7948.398449452414
p[160] 7950.521555078032
p[161] 8310.93804291698
p[162] 8208.607090475292
p

p[873] 46145.88858055651
p[874] 45855.05157476963


In [ ]:
for constr in whole_x.getConstrs():
    print(constr.Pi)

## ②前半と後半をつなげるveb.
1. 適当なところで前後に分ける
1. 前半後半それぞれのPrimalを解く
1. 前半と後半それぞれのPrimalの最適解を、全体のDualに入れて解く
1. 全体のPrimalの最適解を得る

### 1. 適当なところで前後に分ける
- ひとまず半分くらいで分けることにする

In [12]:
threshold = len(tour)//2
#print(threshold)

former = tour[:threshold]
latter = tour[threshold:]
print(f"巡回路全体は、{tour}")
print(f"前半は、{former}")
print(f"後半は、{latter}")

巡回路全体は、[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

### 2. 前半後半それぞれのPrimalを解く

In [13]:
import numpy as np
# 係数行列、ベクトルを整える
A1x, A1p, A2x, A2p, A3x, A3p, b1, b2, b3, c1, c2 = [], [], [], [], [], [], [], [], [], [], []
## A1, A2, A3を定める
## b1, b2, b3を定める
for Ax_i, Ap_i, b_i in zip(Ax, Ap, b):
    if np.linalg.norm(Ax_i[threshold:], ord=2)==0.:
        A1x.append(Ax_i[:threshold])
        A1p.append(Ap_i[:threshold])
        b1.append(b_i)
    elif np.linalg.norm(Ax_i[:threshold], ord=2)==0.:
        A2x.append(Ax_i[threshold:])
        A2p.append(Ap_i[threshold:])
        b2.append(b_i)
    else:
        A3x.append(Ax_i)
        A3p.append(Ap_i)
        b3.append(b_i)
## c1, c2, c3を定める
c1 = c[:threshold]
c2 = c[threshold:]
## リストからnumpy arrayに変換
A1x = np.array(A1x)
A1p = np.array(A1p)
A2x = np.array(A2x)
A2p = np.array(A2p)
A3x = np.array(A3x)
A3p = np.array(A3p)
b1 = np.array(b1)
b2 = np.array(b2)
b3 = np.array(b3)
c1 = np.array(c1)
c2 = np.array(c2)
for i in range(1, 4):
    print(f"A{i}x : ", end="")
    print(eval("A"+str(i)+"x"))
    print(f"A{i}p : ", end="")
    print(eval("A"+str(i)+"p"))
    print(f"b{i} : ", end="")
    print(eval("b"+str(i)))
    if i <= 2:
        print(eval("c"+str(i)))

A1x : [[ 1 -1  0 ...  0  0  0]
 [ 0  1 -1 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0  1]]
A1p : [[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0 -1]]
b1 : [-35.77708764 -72.23572523 -28.23118843 ... 122.         -50.
 328.        ]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [14]:
# Gurobiによって最適解を求める
former_x = solve_primal(A1x, A1p, b1, c1, instance_name+"former_x", len(former))
latter_x = solve_primal(A2x, A2p, b2, c2, instance_name+"latter_x", len(latter))

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 1310 rows, 874 columns and 2620 nonzeros
Model fingerprint: 0x958d4c8d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 1292 rows and 857 columns
Presolve time: 0.02s
Presolved: 18 rows, 17 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.8534314e+06   5.955573e+01   0.000000e+00      0s
       2    4.8535107e+06   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds
Optimal objective  4.853510652e+06
車両が顧客0に到着する時刻は、[0.]
車両が顧客1に到着する時刻は、[35.77708764]
車両が顧客2に到着する時刻は、[108.01281287]
車両が顧客3に到着する時刻は、[136.2440013]
車両が顧客4に到着する時刻は、[175.53776671]
車両が顧客5に到着する時刻は、[221.07895686]
車両が顧客6に到着する時刻は、[235.22109248]
車両が顧客7に到着する時刻は、[253.57865223]
車両が顧客8に到着する時刻は、[323.81397098]
車両が顧客9に到着する時刻は、[355.21460792]
車両が顧客10に到着する時刻は、[396.939900

Optimize a model with 1313 rows, 876 columns and 2626 nonzeros
Model fingerprint: 0x5eaede05
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 1298 rows and 862 columns
Presolve time: 0.01s
Presolved: 15 rows, 14 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9356980e+06   1.033067e+02   0.000000e+00      0s
       4    4.9358086e+06   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective  4.935808604e+06
車両が顧客0に到着する時刻は、[0.]
車両が顧客1に到着する時刻は、[26.41968963]
車両が顧客2に到着する時刻は、[101.65930685]
車両が顧客3に到着する時刻は、[166.5052782]
車両が顧客4に到着する時刻は、[216.4852742]
車両が顧客5に到着する時刻は、[260.32589463]
車両が顧客6に到着する時刻は、[336.74578733]
車両が顧客7に到着する時刻は、[417.73961425]
車両が顧客8に到着する時刻は、[475.99766091]
車両が顧客9に到着する時刻は、[564.28129473]
車両が顧客10に到着する時刻は、[611.4087814]
車両が顧客11に到着する時刻は、[660.50053223]
車両が顧客12に到着する時刻は、[709.7

In [ ]:
for v in former_x.getVars():
        print('%s %g %g' % (v.varName, v.x, v.VBasis))

In [ ]:
for v in latter_x.getVars():
        print('%s %g %g' % (v.varName, v.x, v.VBasis))

In [ ]:
for constr in former_x.getConstrs()+latter_x.getConstrs():
    print(constr.Pi, constr.CBasis)

### 3. 前半後半それぞれのPrimalの最適解を、全体のDualに入れて解く

In [15]:
# 初期解の保存
basis_like = np.array([constr.Pi for constr in former_x.getConstrs()+latter_x.getConstrs()])
y3 = np.zeros((Ax.shape[0]-basis_like.shape[0],))
basis_like = np.append(basis_like, y3)

In [ ]:
print(f"Ax.T.shape={Ax.T.shape}\t\t\tAp.T.shape={Ap.T.shape}")
for Ax_i, Ap_i in zip(Ax.T, Ap.T):
    print(Ax_i, "\t", Ap_i)

In [16]:
# Gurobiによって最適解を求める
whole_y = solve_dual(Ax, Ap, b, c, instance_name+"whole_y", basis_like.shape[0], basis_like)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 4374 rows, 2624 columns and 7872 nonzeros
Model fingerprint: 0xbf1ee39f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4370 rows and 2606 columns
Presolve time: 0.02s
Presolved: 4 rows, 18 columns, 23 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5777088e+31   1.000000e+30   3.577709e+01      0s
       1    2.0056287e+07   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.03 seconds
Optimal objective  2.005628700e+07
主問題における制約0の潜在価格は、[859.]
主問題における制約1の潜在価格は、[860.]
主問題における制約2の潜在価格は、[860.]
主問題における制約3の潜在価格は、[860.]
主問題における制約4の潜在価格は、[860.]
主問題における制約5の潜在価格は、[860.]
主問題における制約6の潜在価格は、[859.]
主問題における制約7の潜在価格は、[859.]
主問題における制約8の潜在価格は、[859.]
主問題における制約9の潜在価格は、[859.]
主問題における制約10の潜在価格は、[859.]
主問題における制約11の潜在価格は、[859.]
主問題における制約12の潜在価格は、[859.]

主問題における制約713の潜在価格は、[161.]
主問題における制約714の潜在価格は、[160.]
主問題における制約715の潜在価格は、[159.]
主問題における制約716の潜在価格は、[158.]
主問題における制約717の潜在価格は、[157.]
主問題における制約718の潜在価格は、[156.]
主問題における制約719の潜在価格は、[155.]
主問題における制約720の潜在価格は、[154.]
主問題における制約721の潜在価格は、[153.]
主問題における制約722の潜在価格は、[152.]
主問題における制約723の潜在価格は、[151.]
主問題における制約724の潜在価格は、[150.]
主問題における制約725の潜在価格は、[149.]
主問題における制約726の潜在価格は、[148.]
主問題における制約727の潜在価格は、[147.]
主問題における制約728の潜在価格は、[146.]
主問題における制約729の潜在価格は、[145.]
主問題における制約730の潜在価格は、[144.]
主問題における制約731の潜在価格は、[143.]
主問題における制約732の潜在価格は、[142.]
主問題における制約733の潜在価格は、[141.]
主問題における制約734の潜在価格は、[140.]
主問題における制約735の潜在価格は、[139.]
主問題における制約736の潜在価格は、[138.]
主問題における制約737の潜在価格は、[137.]
主問題における制約738の潜在価格は、[136.]
主問題における制約739の潜在価格は、[135.]
主問題における制約740の潜在価格は、[134.]
主問題における制約741の潜在価格は、[133.]
主問題における制約742の潜在価格は、[132.]
主問題における制約743の潜在価格は、[131.]
主問題における制約744の潜在価格は、[130.]
主問題における制約745の潜在価格は、[129.]
主問題における制約746の潜在価格は、[128.]
主問題における制約747の潜在価格は、[127.]
主問題における制約748の潜在価格は、[126.]
主問題における制約749の潜在価格は、[125.]
主問題における制約750の潜在価格は、[124.]
主問題における制約751

主問題における制約1203の潜在価格は、[1.]
主問題における制約1204の潜在価格は、[0.]
主問題における制約1205の潜在価格は、[1.]
主問題における制約1206の潜在価格は、[0.]
主問題における制約1207の潜在価格は、[1.]
主問題における制約1208の潜在価格は、[0.]
主問題における制約1209の潜在価格は、[1.]
主問題における制約1210の潜在価格は、[0.]
主問題における制約1211の潜在価格は、[1.]
主問題における制約1212の潜在価格は、[0.]
主問題における制約1213の潜在価格は、[1.]
主問題における制約1214の潜在価格は、[0.]
主問題における制約1215の潜在価格は、[1.]
主問題における制約1216の潜在価格は、[0.]
主問題における制約1217の潜在価格は、[1.]
主問題における制約1218の潜在価格は、[0.]
主問題における制約1219の潜在価格は、[1.]
主問題における制約1220の潜在価格は、[0.]
主問題における制約1221の潜在価格は、[1.]
主問題における制約1222の潜在価格は、[0.]
主問題における制約1223の潜在価格は、[1.]
主問題における制約1224の潜在価格は、[0.]
主問題における制約1225の潜在価格は、[1.]
主問題における制約1226の潜在価格は、[0.]
主問題における制約1227の潜在価格は、[1.]
主問題における制約1228の潜在価格は、[0.]
主問題における制約1229の潜在価格は、[1.]
主問題における制約1230の潜在価格は、[0.]
主問題における制約1231の潜在価格は、[1.]
主問題における制約1232の潜在価格は、[0.]
主問題における制約1233の潜在価格は、[1.]
主問題における制約1234の潜在価格は、[0.]
主問題における制約1235の潜在価格は、[1.]
主問題における制約1236の潜在価格は、[0.]
主問題における制約1237の潜在価格は、[1.]
主問題における制約1238の潜在価格は、[0.]
主問題における制約1239の潜在価格は、[1.]
主問題における制約1240の潜在価格は、[0.]
主問題における制約1241の潜在価格は、[1.]
主問題における制約1242の潜在価格は、[0.]


主問題における制約1768の潜在価格は、[0.]
主問題における制約1769の潜在価格は、[1.]
主問題における制約1770の潜在価格は、[0.]
主問題における制約1771の潜在価格は、[1.]
主問題における制約1772の潜在価格は、[0.]
主問題における制約1773の潜在価格は、[1.]
主問題における制約1774の潜在価格は、[0.]
主問題における制約1775の潜在価格は、[1.]
主問題における制約1776の潜在価格は、[0.]
主問題における制約1777の潜在価格は、[1.]
主問題における制約1778の潜在価格は、[0.]
主問題における制約1779の潜在価格は、[1.]
主問題における制約1780の潜在価格は、[0.]
主問題における制約1781の潜在価格は、[1.]
主問題における制約1782の潜在価格は、[0.]
主問題における制約1783の潜在価格は、[1.]
主問題における制約1784の潜在価格は、[0.]
主問題における制約1785の潜在価格は、[1.]
主問題における制約1786の潜在価格は、[0.]
主問題における制約1787の潜在価格は、[1.]
主問題における制約1788の潜在価格は、[0.]
主問題における制約1789の潜在価格は、[1.]
主問題における制約1790の潜在価格は、[0.]
主問題における制約1791の潜在価格は、[1.]
主問題における制約1792の潜在価格は、[0.]
主問題における制約1793の潜在価格は、[1.]
主問題における制約1794の潜在価格は、[0.]
主問題における制約1795の潜在価格は、[1.]
主問題における制約1796の潜在価格は、[0.]
主問題における制約1797の潜在価格は、[1.]
主問題における制約1798の潜在価格は、[0.]
主問題における制約1799の潜在価格は、[1.]
主問題における制約1800の潜在価格は、[0.]
主問題における制約1801の潜在価格は、[1.]
主問題における制約1802の潜在価格は、[0.]
主問題における制約1803の潜在価格は、[1.]
主問題における制約1804の潜在価格は、[0.]
主問題における制約1805の潜在価格は、[1.]
主問題における制約1806の潜在価格は、[0.]
主問題における制約1807の潜在価格は、[1.]


主問題における制約2124の潜在価格は、[0.]
主問題における制約2125の潜在価格は、[1.]
主問題における制約2126の潜在価格は、[0.]
主問題における制約2127の潜在価格は、[1.]
主問題における制約2128の潜在価格は、[0.]
主問題における制約2129の潜在価格は、[1.]
主問題における制約2130の潜在価格は、[0.]
主問題における制約2131の潜在価格は、[1.]
主問題における制約2132の潜在価格は、[0.]
主問題における制約2133の潜在価格は、[1.]
主問題における制約2134の潜在価格は、[0.]
主問題における制約2135の潜在価格は、[1.]
主問題における制約2136の潜在価格は、[0.]
主問題における制約2137の潜在価格は、[1.]
主問題における制約2138の潜在価格は、[0.]
主問題における制約2139の潜在価格は、[1.]
主問題における制約2140の潜在価格は、[0.]
主問題における制約2141の潜在価格は、[1.]
主問題における制約2142の潜在価格は、[0.]
主問題における制約2143の潜在価格は、[1.]
主問題における制約2144の潜在価格は、[0.]
主問題における制約2145の潜在価格は、[1.]
主問題における制約2146の潜在価格は、[0.]
主問題における制約2147の潜在価格は、[1.]
主問題における制約2148の潜在価格は、[0.]
主問題における制約2149の潜在価格は、[1.]
主問題における制約2150の潜在価格は、[0.]
主問題における制約2151の潜在価格は、[1.]
主問題における制約2152の潜在価格は、[0.]
主問題における制約2153の潜在価格は、[1.]
主問題における制約2154の潜在価格は、[0.]
主問題における制約2155の潜在価格は、[1.]
主問題における制約2156の潜在価格は、[0.]
主問題における制約2157の潜在価格は、[1.]
主問題における制約2158の潜在価格は、[0.]
主問題における制約2159の潜在価格は、[1.]
主問題における制約2160の潜在価格は、[0.]
主問題における制約2161の潜在価格は、[1.]
主問題における制約2162の潜在価格は、[0.]
主問題における制約2163の潜在価格は、[1.]


主問題における制約2482の潜在価格は、[0.]
主問題における制約2483の潜在価格は、[1.]
主問題における制約2484の潜在価格は、[0.]
主問題における制約2485の潜在価格は、[1.]
主問題における制約2486の潜在価格は、[0.]
主問題における制約2487の潜在価格は、[1.]
主問題における制約2488の潜在価格は、[0.]
主問題における制約2489の潜在価格は、[1.]
主問題における制約2490の潜在価格は、[0.]
主問題における制約2491の潜在価格は、[1.]
主問題における制約2492の潜在価格は、[0.]
主問題における制約2493の潜在価格は、[1.]
主問題における制約2494の潜在価格は、[0.]
主問題における制約2495の潜在価格は、[1.]
主問題における制約2496の潜在価格は、[0.]
主問題における制約2497の潜在価格は、[1.]
主問題における制約2498の潜在価格は、[0.]
主問題における制約2499の潜在価格は、[1.]
主問題における制約2500の潜在価格は、[0.]
主問題における制約2501の潜在価格は、[1.]
主問題における制約2502の潜在価格は、[0.]
主問題における制約2503の潜在価格は、[1.]
主問題における制約2504の潜在価格は、[0.]
主問題における制約2505の潜在価格は、[1.]
主問題における制約2506の潜在価格は、[0.]
主問題における制約2507の潜在価格は、[1.]
主問題における制約2508の潜在価格は、[0.]
主問題における制約2509の潜在価格は、[1.]
主問題における制約2510の潜在価格は、[0.]
主問題における制約2511の潜在価格は、[1.]
主問題における制約2512の潜在価格は、[0.]
主問題における制約2513の潜在価格は、[1.]
主問題における制約2514の潜在価格は、[0.]
主問題における制約2515の潜在価格は、[1.]
主問題における制約2516の潜在価格は、[0.]
主問題における制約2517の潜在価格は、[1.]
主問題における制約2518の潜在価格は、[0.]
主問題における制約2519の潜在価格は、[1.]
主問題における制約2520の潜在価格は、[0.]
主問題における制約2521の潜在価格は、[1.]


In [17]:
# 最適解
for var in whole_y.getVars():
    print(var.varName, var.X)

y[0] 859.0
y[1] 860.0
y[2] 860.0
y[3] 860.0
y[4] 860.0
y[5] 860.0
y[6] 859.0
y[7] 859.0
y[8] 859.0
y[9] 859.0
y[10] 859.0
y[11] 859.0
y[12] 859.0
y[13] 859.0
y[14] 858.0
y[15] 857.0
y[16] 857.0
y[17] 857.0
y[18] 856.0
y[19] 855.0
y[20] 854.0
y[21] 853.0
y[22] 852.0
y[23] 851.0
y[24] 850.0
y[25] 849.0
y[26] 848.0
y[27] 847.0
y[28] 846.0
y[29] 845.0
y[30] 844.0
y[31] 843.0
y[32] 842.0
y[33] 841.0
y[34] 840.0
y[35] 839.0
y[36] 838.0
y[37] 837.0
y[38] 836.0
y[39] 835.0
y[40] 834.0
y[41] 833.0
y[42] 832.0
y[43] 831.0
y[44] 830.0
y[45] 829.0
y[46] 828.0
y[47] 827.0
y[48] 826.0
y[49] 825.0
y[50] 824.0
y[51] 823.0
y[52] 822.0
y[53] 821.0
y[54] 820.0
y[55] 819.0
y[56] 818.0
y[57] 817.0
y[58] 816.0
y[59] 815.0
y[60] 814.0
y[61] 813.0
y[62] 812.0
y[63] 811.0
y[64] 810.0
y[65] 809.0
y[66] 808.0
y[67] 807.0
y[68] 806.0
y[69] 805.0
y[70] 804.0
y[71] 803.0
y[72] 802.0
y[73] 801.0
y[74] 800.0
y[75] 799.0
y[76] 798.0
y[77] 797.0
y[78] 796.0
y[79] 795.0
y[80] 794.0
y[81] 793.0
y[82] 792.0
y[83] 791.0
y[

y[900] 0.0
y[901] 0.0
y[902] 0.0
y[903] 1.0
y[904] 0.0
y[905] 1.0
y[906] 0.0
y[907] 0.0
y[908] 0.0
y[909] 0.0
y[910] 0.0
y[911] 1.0
y[912] 0.0
y[913] 1.0
y[914] 0.0
y[915] 1.0
y[916] 0.0
y[917] 1.0
y[918] 0.0
y[919] 1.0
y[920] 0.0
y[921] 1.0
y[922] 0.0
y[923] 1.0
y[924] 0.0
y[925] 1.0
y[926] 0.0
y[927] 1.0
y[928] 0.0
y[929] 1.0
y[930] 0.0
y[931] 1.0
y[932] 0.0
y[933] 1.0
y[934] 0.0
y[935] 1.0
y[936] 0.0
y[937] 1.0
y[938] 0.0
y[939] 1.0
y[940] 0.0
y[941] 1.0
y[942] 0.0
y[943] 1.0
y[944] 0.0
y[945] 1.0
y[946] 0.0
y[947] 1.0
y[948] 0.0
y[949] 1.0
y[950] 0.0
y[951] 1.0
y[952] 0.0
y[953] 1.0
y[954] 0.0
y[955] 1.0
y[956] 0.0
y[957] 1.0
y[958] 0.0
y[959] 1.0
y[960] 0.0
y[961] 1.0
y[962] 0.0
y[963] 1.0
y[964] 0.0
y[965] 1.0
y[966] 0.0
y[967] 1.0
y[968] 0.0
y[969] 1.0
y[970] 0.0
y[971] 1.0
y[972] 0.0
y[973] 1.0
y[974] 0.0
y[975] 1.0
y[976] 0.0
y[977] 1.0
y[978] 0.0
y[979] 1.0
y[980] 0.0
y[981] 1.0
y[982] 0.0
y[983] 1.0
y[984] 0.0
y[985] 1.0
y[986] 0.0
y[987] 1.0
y[988] 0.0
y[989] 1.0
y[990] 0.0

y[1650] 0.0
y[1651] 1.0
y[1652] 0.0
y[1653] 1.0
y[1654] 0.0
y[1655] 1.0
y[1656] 0.0
y[1657] 1.0
y[1658] 0.0
y[1659] 1.0
y[1660] 0.0
y[1661] 1.0
y[1662] 0.0
y[1663] 1.0
y[1664] 0.0
y[1665] 1.0
y[1666] 0.0
y[1667] 1.0
y[1668] 0.0
y[1669] 1.0
y[1670] 0.0
y[1671] 1.0
y[1672] 0.0
y[1673] 1.0
y[1674] 0.0
y[1675] 1.0
y[1676] 0.0
y[1677] 1.0
y[1678] 0.0
y[1679] 1.0
y[1680] 0.0
y[1681] 1.0
y[1682] 0.0
y[1683] 1.0
y[1684] 0.0
y[1685] 1.0
y[1686] 0.0
y[1687] 1.0
y[1688] 0.0
y[1689] 1.0
y[1690] 0.0
y[1691] 1.0
y[1692] 0.0
y[1693] 1.0
y[1694] 0.0
y[1695] 1.0
y[1696] 0.0
y[1697] 1.0
y[1698] 0.0
y[1699] 1.0
y[1700] 0.0
y[1701] 1.0
y[1702] 0.0
y[1703] 1.0
y[1704] 0.0
y[1705] 1.0
y[1706] 0.0
y[1707] 1.0
y[1708] 0.0
y[1709] 1.0
y[1710] 0.0
y[1711] 1.0
y[1712] 0.0
y[1713] 1.0
y[1714] 0.0
y[1715] 1.0
y[1716] 0.0
y[1717] 1.0
y[1718] 0.0
y[1719] 1.0
y[1720] 0.0
y[1721] 1.0
y[1722] 0.0
y[1723] 1.0
y[1724] 0.0
y[1725] 1.0
y[1726] 0.0
y[1727] 1.0
y[1728] 0.0
y[1729] 1.0
y[1730] 0.0
y[1731] 1.0
y[1732] 0.0
y[17

### 4. 全体のPrimalの最適解を得る

In [18]:
for i, constr in enumerate(whole_y.getConstrs()):
    print(constr.Pi)

0.0
-35.77708763999664
-108.01281287342944
-136.24400130041565
-175.53776670919265
-221.07895685613545
-235.2210924798664
-253.57865223055222
-323.8139709811637
-355.2146079173789
-396.939900007429
-451.4284309107301
-483.6774619039243
-574.682956243552
-669.2398174435727
-718.2806167845297
-771.5065503132147
-798.4137984073622
-866.2149783380978
-896.2981962510804
-899.9037475265444
-980.2654299380022
-1003.9719691202615
-1015.6338729099522
-1028.6722777203574
-1076.2117350163762
-1109.9459906032423
-1166.8318412165538
-1230.9021151351192
-1344.7133623824805
-1440.2382282551944
-1442.474296232694
-1514.585321741974
-1566.9112235497785
-1581.0533591735093
-1657.2110902321483
-1717.7503337028095
-1764.9520285876333
-1816.9231744283784
-1883.2104281050847
-1930.917869872591
-2020.4723225815433
-2076.0510955079044
-2149.3995787914732
-2165.555073212877
-2177.596667791669
-2203.6158914541843
-2249.7136137406487
-2288.930956842904
-2315.1797663397174
-2362.6981838217844
-2436.7387132634353


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [19]:
for var in whole_x.getVars():
    print(var.varName, var.X)

x[0] 0.0
x[1] 35.77708763999664
x[2] 108.01281287342944
x[3] 136.24400130041565
x[4] 175.53776670919265
x[5] 221.07895685613545
x[6] 235.2210924798664
x[7] 253.57865223055222
x[8] 323.8139709811637
x[9] 355.2146079173789
x[10] 396.939900007429
x[11] 451.4284309107301
x[12] 483.6774619039243
x[13] 574.682956243552
x[14] 669.2398174435727
x[15] 718.2806167845297
x[16] 771.5065503132147
x[17] 798.4137984073622
x[18] 866.2149783380978
x[19] 896.2981962510804
x[20] 899.9037475265444
x[21] 980.2654299380022
x[22] 1003.9719691202615
x[23] 1015.6338729099522
x[24] 1028.6722777203574
x[25] 1076.2117350163762
x[26] 1109.9459906032423
x[27] 1166.8318412165538
x[28] 1230.9021151351192
x[29] 1344.7133623824805
x[30] 1440.2382282551944
x[31] 1442.474296232694
x[32] 1514.585321741974
x[33] 1566.9112235497785
x[34] 1581.0533591735093
x[35] 1657.2110902321483
x[36] 1717.7503337028095
x[37] 1764.9520285876333
x[38] 1816.9231744283784
x[39] 1883.2104281050847
x[40] 1930.917869872591
x[41] 2020.4723225815

x[832] 44571.012527622814
x[833] 44617.186113145275
x[834] 44677.59306638743
x[835] 44688.3633960017
x[836] 44772.37529992014
x[837] 44858.38111367712
x[838] 44901.9471554987
x[839] 44971.954297991455
x[840] 45028.03459756269
x[841] 45077.065200251214
x[842] 45115.340518669225
x[843] 45127.382113248015
x[844] 45207.66287059372
x[845] 45274.8715009919
x[846] 45319.42483981179
x[847] 45410.64435526764
x[848] 45500.53317548462
x[849] 45520.75692390078
x[850] 45569.7875265893
x[851] 45614.56475294378
x[852] 45652.00804339747
x[853] 45715.158657012864
x[854] 45758.170283348074
x[855] 45829.34612194517
x[856] 45856.80518238066
x[857] 45911.88693979634
x[858] 45998.26402008967
x[859] 46016.374790365946
x[860] 46080.32789818722
x[861] 46156.130272820206
x[862] 46241.99644776704
x[863] 46278.12123150341
x[864] 46334.02293094091
x[865] 46408.02293094091
x[866] 46476.000868538344
x[867] 46531.71442164708
x[868] 46572.23602082579
x[869] 46589.92782683874
x[870] 46642.014293411165
x[871] 46686.9587

p[707] 37957.33278884072
p[708] 37391.768593078814
p[709] 37141.15326002043
p[710] 37750.25227953402
p[711] 38046.96739903539
p[712] 37965.05540652603
p[713] 38134.47777876003
p[714] 38233.13411952756
p[715] 37893.393526226784
p[716] 37967.1468032999
p[717] 37674.072441613
p[718] 38122.072441613
p[719] 38313.19554723862
p[720] 37583.224952645396
p[721] 38228.224952645396
p[722] 38140.902284613745
p[723] 38133.01586234147
p[724] 37961.09426358287
p[725] 38833.74349458983
p[726] 38632.2242616318
p[727] 38764.51863331469
p[728] 38777.24171623801
p[729] 38373.59587977561
p[730] 39014.4196663388
p[731] 38686.25045060666
p[732] 38448.870569472754
p[733] 38765.471644710495
p[734] 39370.885626725874
p[735] 39189.37091942976
p[736] 39550.95523164725
p[737] 39409.925105698014
p[738] 39395.06528420666
p[739] 39388.419446088454
p[740] 39308.18886539435
p[741] 39820.63883594794
p[742] 39708.53833088455
p[743] 39448.34000449542
p[744] 39984.034202450675
p[745] 39227.68945305664
p[746] 39802.67429806